In [1]:
#Importing the necessary packages
import pandas as pd
import numpy as np
import glob
import time

In [2]:
#Reading the wnv data
wnv = pd.read_csv("WNV_NI_NNI_1999to2015_prevalence_incidence_final_20180530.csv", encoding="ISO-8859-1")
print("Shape of the Dataframe: ", wnv.shape, "\n")
wnv.head(5)

Shape of the Dataframe:  (3108, 153) 



,GEOID10,Select_County,STATEFP10,STATENS,STUSPS,STNAME,COUNTYFP10,COUNTYNS10,CTYNAME,POPESTIMATE1999,...,NIIR_2011_Z,NIIR_2012_Z,NIIR_2013_Z,NIIR_2014_Z,NIIR_2015_Z,NIIR_EST_AVG_Z,NIIR_EST_MED_Z,County_WNV_Class,County_WNV_Class2,filter_$
0,1001,1,1,1779775,AL,Alabama,1,161526,Autauga County,42963,...,-.172668378274114,-.313041876784145,-.237005866057391,-.211466937746909,-.197767338621807,-.230659201920693,-.231569016088473,2,NI-S,0
1,1003,1,1,1779775,AL,Alabama,3,161527,Baldwin County,137555,...,-.172668378274114,.0148439683883608,-.177634573907632,-.211466937746909,-.197767338621807,-.177866829768146,-.186748999098691,4,NI-M-pre12,0
2,1005,0,1,1779775,AL,Alabama,5,161528,Barbour County,28866,...,-.172668378274114,-.313041876784145,-.237005866057391,-.211466937746909,-.197767338621807,-.220130592240443,-.221517976917691,2,NI-S-NNI,0
3,1007,0,1,1779775,AL,Alabama,7,161529,Bibb County,20560,...,,,,,,,,3,NI-S,0
4,1009,0,1,1779775,AL,Alabama,9,161530,Blount County,50237,...,,,,,,,,1,NNI-S,0


In [3]:
print(list(wnv.columns), "\n")

['GEOID10', 'Select_County', 'STATEFP10', 'STATENS', 'STUSPS', 'STNAME', 'COUNTYFP10', 'COUNTYNS10', 'CTYNAME', 'POPESTIMATE1999', 'POPESTIMATE2000', 'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 'Note', 'CDC_WNV_Record', 'CDC_NI_Record', 'CDC_NNI_Record', 'NI_Total', 'Years_with_NI', 'NI_1999', 'NI_2000', 'NI_2001', 'NI_2002', 'NI_2003', 'NI_2004', 'NI_2005', 'NI_2006', 'NI_2007', 'NI_2008', 'NI_2009', 'NI_2010', 'NI_2011', 'NI_2012', 'NI_2013', 'NI_2014', 'NI_2015', 'NNI_Total', 'Years_with_NNI', 'NNI_1999', 'NNI_2000', 'NNI_2001', 'NNI_2002', 'NNI_2003', 'NNI_2004', 'NNI_2005', 'NNI_2006', 'NNI_2007', 'NNI_2008', 'NNI_2009', 'NNI_2010', 'NNI_2011', 'NNI_2012', 'NNI_2013', 'NNI_2014', 'NNI_2015', 'CDC_WNV_note', 'NI_IR_1999', 'NI_IR_2000', 'NI_IR_20

In [4]:
#Selecting only urban counties
wnv = wnv[wnv['Select_County'] == 1]
print("Shape of the DataFrame now: ", wnv.shape)

df = pd.DataFrame(columns = ['YEAR','GEOID10','CTYNAME','STUSPS','DeG12Region','Total_BG.sBG','County_WNV_Class','County_WNV_Class2','POPESTIMATE','NI_','NNI_','NI_IR_', 'Establish_', 'NIIR_Z'])

#### df = pd.DataFrame(columns = ['YEAR','GEOID10','CTYNAME','STUSPS','DeG12Region', 'Total_BG.sBG','County_WNV_Class','County_WNV_Class2', 'POPESTIMATE', 'NI_', 'NNI_', 'NI_IR_', 'Establish_','NIIR_Z', CDC_WNV_Record', 
####     'CDC_NI_Record', 'CDC_NNI_Record', 'NI_Total', 'Years_with_NI', 'NNI_Total', 'Years_with_NNI', 'MEAN_ANNUAL_IR_ESTAB',
####     'CUM_IR_1999to2015', 'MEAN_ANNUAL_IR_1999to2015', 'POPPCT_URB', 'AREA_URBAN', 'POPPCT_RUR', 'AREA_RURAL',
####     'AREAPCT_RU', 'NIIR_EST_AVG_Z', 'NIIR_EST_MED_Z'])


year_list = ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
                '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015' ]

Shape of the DataFrame now:  (852, 153)


## Dropping the following columns: 

#### 1. Note - Only one row has data, so dropping it
#### 2. CDC_WNV_Record, CDC_NI_Record, CDC_NNI_Record and a few more - 
####     - Already have these values for every year, these columns represent the aggregated values
#### 3. CDC_WNV_note - No data
#### 4. MEAN_ANNUAL_IR_1999to2004, MEAN_ANNUAL_IR_2005to2015, MEAN_ANNUAL_IR_2006to2015, NI_Total_2006to2015,
####   'NI_Total_2005to2015' - The combined data of these columns have been added already
#### 5. 'ST_ABBREV', 'NAMELSAD10' - Repeated columns
#### 6. 'POP_URBAN', 'POP_RURAL', 'POPDEN_URB', 'POPDEN_RUR' - We have population columns as per year and these are aggregated values
#### 7. 'NCHS_2013', 'NCHS_2006', 'NCHS_1990', 'NCHS_2013_3class' -> Not present for all years, so dropping it for now
#### 9. PCT_CTYPOP_sBG, NI_NNI_DIFF, filter_ -> Not sure what these columns are, dropping it for now.

#### 10. Year_1st_NI_case, Year_1st_NNI_case -> Should we remove all the rows for those counties before the first year?

In [5]:
for  year in year_list:
    
    #Starting by creating one column only
    temp_df = pd.DataFrame(columns = ['POPESTIMATE'])
    
    #All yearly data is being mapped to the corresponding year row
    POP = 'POPESTIMATE' + year
    temp_df.loc[:,'POPESTIMATE'] = wnv[POP]
    
    NI = 'NI_' + year
    temp_df.loc[:,'NI_'] = wnv[NI]
    
    NNI = 'NNI_' + year
    temp_df.loc[:,'NNI_'] = wnv[NNI]
    
    NI_IR = 'NI_IR_' + year
    temp_df.loc[:,'NI_IR_'] = wnv[NI_IR]
    
    #There is no data for Establish and NIIR_Z columns for the year 1999 
    if (year == '1999'): 
        temp_df.loc[:,'Establish_'] = 0
        temp_df.loc[:,'NIIR_Z'] = 0

    else:
        Establish = 'Establish_' + year
        temp_df.loc[:,'Establish_'] = wnv[Establish]
        
        NIIR_Z = 'NIIR_' + year + '_Z'
        temp_df.loc[:,'NIIR_Z'] = wnv[NIIR_Z]
    
    temp_df.insert(0, 'YEAR', year)
    
    #Common for each temp_df(common for each year)
    temp_df.insert(1, 'GEOID10', wnv['GEOID10'])
    temp_df.insert(2, 'CTYNAME', wnv['CTYNAME'])
    temp_df.insert(3, 'STUSPS', wnv['STUSPS'])
    temp_df.insert(4, 'DeG12Region', wnv['DeG12Region'])
    temp_df.insert(5, 'Total_BG.sBG', wnv['Total_BG.sBG'])
    temp_df.insert(6, 'County_WNV_Class', wnv['County_WNV_Class'])
    temp_df.insert(7, 'County_WNV_Class2', wnv['County_WNV_Class2'])
       
    
    ####temp_df.insert(14, 'CDC_WNV_Record', wnv['CDC_WNV_Record'])
    ####temp_df.insert(15, 'CDC_NI_Record', wnv['CDC_NI_Record'])
    ####temp_df.insert(16, 'CDC_NNI_Record', wnv['CDC_NNI_Record'])
    ####temp_df.insert(17, 'NI_Total', wnv['NI_Total'])
    ####temp_df.insert(18, 'Years_with_NI', wnv['Years_with_NI'])
    ####temp_df.insert(19, 'NNI_Total', wnv['NNI_Total'])
    ####temp_df.insert(20, 'Years_with_NNI', wnv['Years_with_NNI'])
    ####temp_df.insert(21, 'MEAN_ANNUAL_IR_ESTAB', wnv['MEAN_ANNUAL_IR_ESTAB'])
    ####temp_df.insert(22, 'CUM_IR_1999to2015', wnv['CUM_IR_1999to2015'])
    ####temp_df.insert(23, 'MEAN_ANNUAL_IR_1999to2015', wnv['MEAN_ANNUAL_IR_1999to2015'])
    ####temp_df.insert(24, 'POPPCT_URB', wnv['POPPCT_URB'])
    ####temp_df.insert(25, 'AREA_URBAN', wnv['AREA_URBAN'])
    ####temp_df.insert(26, 'AREAPCT_UR', wnv['AREAPCT_UR'])
    ####temp_df.insert(27, 'POPPCT_RUR', wnv['POPPCT_RUR'])
    ####temp_df.insert(28, 'AREA_RURAL', wnv['AREA_RURAL'])
    ####temp_df.insert(29, 'AREAPCT_RU', wnv['AREAPCT_RU'])
    ####temp_df.insert(30, 'NIIR_EST_AVG_Z', wnv['NIIR_EST_AVG_Z'])
    ####temp_df.insert(31, 'NIIR_EST_MED_Z', wnv['NIIR_EST_MED_Z'])

    df = df.append(temp_df, ignore_index=True) 


### Adding weather data to the df
#### Temperature, humidity and precipitation are currently present in monthly files. This is aggregated per year and added under one column each for T, H and P

In [6]:
#Adding temperature, humidity and precipitation data
base = "narr_urban_county_data"

In [7]:
#Adding temperature data
column_names_temp = ["min_K", "max_K", "mean_temp", "geoid", "name"]
temperature = pd.DataFrame()

#For all years from 1999-2015
for yr in year_list:
    yearly_files = glob.glob(base + "/air.sfc/" + yr + "*.csv") #12 files per year
    
    for fl_name in yearly_files:
        temp_df2 = pd.read_csv(fl_name, index_col=None, header=0, names=column_names_temp) #Individual files in a temp dataframe
        year = fl_name[-11:-7]
        month = fl_name[-6:-4]
        temp_df2.insert(5, "year", year) #Adding year and month from filename for further use
        temp_df2.insert(6, "month", month)
        
        temperature = temperature.append(temp_df2, ignore_index=True, sort=False)
        temperature = temperature[['year', 'geoid', 'mean_temp']]
        
temperature = temperature.groupby(['year', 'geoid']).mean()
#The temperature dataframe has one column with temperature mean value for every year for all urban counties
print("Temperature Dataframe created: ", temperature.shape)

#Merging the temeperature column to the main df
df = pd.merge(df, temperature,  how='outer', left_on=['GEOID10','YEAR'], right_on = ['geoid','year'])

Temperature Dataframe created:  (14484, 1)


In [8]:
#Adding humidity data
column_names_temp = ["min_hum", "max_hum", "mean_hum", "geoid", "name"]
humidity = pd.DataFrame()

for yr in year_list:
    yearly_files = glob.glob(base + "/rhum.2m/" + yr + "*.csv")
    
    for fl_name in yearly_files:
        temp_df2 = pd.read_csv(fl_name, index_col=None, header=0, names=column_names_temp) #Individual files in a humidity dataframe
        year = fl_name[-11:-7]
        month = fl_name[-6:-4]
        temp_df2.insert(5, "year", year) #Adding year and month from filename for further use
        temp_df2.insert(6, "month", month)
        
        humidity = humidity.append(temp_df2, ignore_index=True, sort=False)
        humidity = humidity[['year', 'geoid', 'mean_hum']]
        
humidity = humidity.groupby(['year', 'geoid']).mean()
print("Humidity Dataframe created: ", humidity.shape)

#Merging the humidity column to the main df
df = pd.merge(df, humidity,  how='outer', left_on=['GEOID10','YEAR'], right_on = ['geoid','year'])

Humidity Dataframe created:  (14484, 1)


In [9]:
#Adding precipitation data
column_names_temp = ["min_prec", "max_prec", "mean_prec", "geoid", "name"]
precipitation = pd.DataFrame()

for yr in year_list:
    yearly_files = glob.glob(base + "/apcp/" + yr + "*.csv")
    
    for fl_name in yearly_files:
        temp_df2 = pd.read_csv(fl_name, index_col=None, header=0, names=column_names_temp) #Individual files in a precipitation dataframe
        year = fl_name[-11:-7]
        month = fl_name[-6:-4]
        temp_df2.insert(5, "year", year) #Adding year and month from filename for further use
        temp_df2.insert(6, "month", month)
        
        precipitation = precipitation.append(temp_df2, ignore_index=True, sort=False)
        precipitation = precipitation[['year', 'geoid', 'mean_prec']]
        
precipitation = precipitation.groupby(['year', 'geoid']).mean()
print("Precipitation Dataframe created: ", precipitation.shape)

#Merging the precipitation column to the main df
df = pd.merge(df, precipitation,  how='outer', left_on=['GEOID10','YEAR'], right_on = ['geoid','year'])

Precipitation Dataframe created:  (14484, 1)


In [10]:
#Adding climate zone column
climate_dict = {'wnc' : ['MT','NE','ND','SD','WY'],
                'enc' : ['IA','MI','MN','WI'],
                'nw' : ['ID','OR','WA'],
                'ne' : ['CT','DE','ME','MD','MA','NH','NJ','NY','PA','RI','VT'],
                'se' : ['AL','FL','GA','NC','SC','VA'],
                'sw' : ['AZ','CO','NM','UT'],
                'c' : ['IL','IN','KY','MO','OH','TN','WV'],
                's' : ['AR','KS','LA','MS','OK','TX'],
                'w' : ['CA','NV']
               }

#The new dictionary swaps keys and values from above dictionary to create the climate zone column
climate_dict2 = {} 
for key, value in climate_dict.items():
    for items in value:
        climate_dict2[items]=key
        
print(climate_dict2)

{'MT': 'wnc', 'NE': 'wnc', 'ND': 'wnc', 'SD': 'wnc', 'WY': 'wnc', 'IA': 'enc', 'MI': 'enc', 'MN': 'enc', 'WI': 'enc', 'ID': 'nw', 'OR': 'nw', 'WA': 'nw', 'CT': 'ne', 'DE': 'ne', 'ME': 'ne', 'MD': 'ne', 'MA': 'ne', 'NH': 'ne', 'NJ': 'ne', 'NY': 'ne', 'PA': 'ne', 'RI': 'ne', 'VT': 'ne', 'AL': 'se', 'FL': 'se', 'GA': 'se', 'NC': 'se', 'SC': 'se', 'VA': 'se', 'AZ': 'sw', 'CO': 'sw', 'NM': 'sw', 'UT': 'sw', 'IL': 'c', 'IN': 'c', 'KY': 'c', 'MO': 'c', 'OH': 'c', 'TN': 'c', 'WV': 'c', 'AR': 's', 'KS': 's', 'LA': 's', 'MS': 's', 'OK': 's', 'TX': 's', 'CA': 'w', 'NV': 'w'}


In [11]:
df['CLIMATE_ZONE'] = df['STUSPS'].map(climate_dict2) #Mapping the values to a dataframe column

In [12]:
df = df.rename(columns={"mean_temp": "TEMPERATURE", "mean_hum": "HUMIDITY", "mean_prec": "PRECIPITATION"})
print(df.shape, "\n")
print(list(df.columns), "\n")
df.head(5)

(14484, 18) 

['YEAR', 'GEOID10', 'CTYNAME', 'STUSPS', 'DeG12Region', 'Total_BG.sBG', 'County_WNV_Class', 'County_WNV_Class2', 'POPESTIMATE', 'NI_', 'NNI_', 'NI_IR_', 'Establish_', 'NIIR_Z', 'TEMPERATURE', 'HUMIDITY', 'PRECIPITATION', 'CLIMATE_ZONE'] 



,YEAR,GEOID10,CTYNAME,STUSPS,DeG12Region,Total_BG.sBG,County_WNV_Class,County_WNV_Class2,POPESTIMATE,NI_,NNI_,NI_IR_,Establish_,NIIR_Z,TEMPERATURE,HUMIDITY,PRECIPITATION,CLIMATE_ZONE
0,1999,1001,Autauga County,AL,2,15,2,NI-S,42963,0,0,0.0,0,0,291.518659,73.042486,3.560286,se
1,1999,1003,Baldwin County,AL,2,28,4,NI-M-pre12,137555,0,0,0.0,0,0,294.361160,75.947582,3.225454,se
2,1999,1015,Calhoun County,AL,2,45,2,NI-S-NNI,114910,0,0,0.0,0,0,290.075280,72.004987,3.033717,se
3,1999,1033,Colbert County,AL,2,24,1,none,54715,0,0,0.0,0,0,290.192782,72.069193,2.872019,se
4,1999,1045,Dale County,AL,2,10,2,NI-S-NNI,49118,0,0,0.0,0,0,292.766317,72.045064,3.138175,se


In [14]:
df.to_csv('wnv_aggr_final.csv', index=False)